<a href="https://colab.research.google.com/github/kochol/tf_tests/blob/master/test02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 02

This is my second test to predict the bitcoin price.

This is a simple model that gets the candle of last 100 hours and try to preedict the next 24 hour price.

This test failed :(

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# enable tf eager execution to support iterating
# tf.enable_eager_execution()

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


## The parameters



In [0]:
# The candle hours that we want to send it as input
in_hours = 100 #@param

# The price prediction that we want to get from our model
out_hours = 24 #@param

# The data we want to use as validation
val_hours = 100 #@param

# The layer nerouns count
nn_count = 6400 #@param

# The data file name to load data from
data_filename = 'binance_last_3m.csv' #@param {type: "string"}

## Load data from csv file

In [3]:
import pandas as pd
import numpy as np

# Download mydata.csv from github
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/kochol/tf_tests/master/{data_filename}

# load all data
mydata = pd.read_csv(data_filename)

print(mydata.columns)
print(mydata.dtypes)
print(mydata.shape)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  182k  100  182k    0     0   825k      0 --:--:-- --:--:-- --:--:--  825k
Index(['year', 'month', 'date', 'hour', 'open', 'high', 'low', 'close', 'vwp',
       'volume', 'trades'],
      dtype='object')
year        int64
month       int64
date        int64
hour        int64
open      float64
high      float64
low       float64
close     float64
vwp       float64
volume    float64
trades      int64
dtype: object
(2274, 11)


## Prepare the data for training

In [4]:
sum_val = val_hours + out_hours
sum_test = sum_val + out_hours

train = pd.DataFrame(
    {       
        'year': mydata[0:-sum_test].year.astype('int32'),
        'month': mydata[0:-sum_test].month.astype('int32'),
        'date': mydata[0:-sum_test].date.astype('int32'),
        'hour': mydata[0:-sum_test].hour.astype('int32'),
        'open': mydata[0:-sum_test].open.astype('int32'),
        'high': mydata[0:-sum_test].high.astype('int32'),
        'low': mydata[0:-sum_test].low.astype('int32'),
        'close': mydata[0:-sum_test].close.astype('int32'),
        'vwp': mydata[0:-sum_test].vwp.astype('int32'),
        'volume': mydata[0:-sum_test].volume.astype('int32'),
        'trades': mydata[0:-sum_test].trades.astype('int32')
    })
print(train.shape)

test = pd.DataFrame({'price': mydata[in_hours:-sum_val].close.astype('int32')})
print(test.shape)

val_in = pd.DataFrame(
    {
        'year': mydata[-sum_val:-out_hours].year.astype('int32'),
        'month': mydata[-sum_val:-out_hours].month.astype('int32'),
        'date': mydata[-sum_val:-out_hours].date.astype('int32'),
        'hour': mydata[-sum_val:-out_hours].hour.astype('int32'),
        'open': mydata[-sum_val:-out_hours].open.astype('int32'),
        'high': mydata[-sum_val:-out_hours].high.astype('int32'),
        'low': mydata[-sum_val:-out_hours].low.astype('int32'),
        'close': mydata[-sum_val:-out_hours].close.astype('int32'),
        'vwp': mydata[-sum_val:-out_hours].vwp.astype('int32'),
        'volume': mydata[-sum_val:-out_hours].volume.astype('int32'),
        'trades': mydata[-sum_val:-out_hours].trades.astype('int32')
    })
print(val_in.shape)

val_test = pd.DataFrame({'price': mydata[-out_hours:].close.astype('int32')})
print(val_test.shape)

(2126, 11)
(2050, 1)
(100, 11)
(24, 1)


## Create the model

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(in_hours, len(mydata.columns))),
  tf.keras.layers.Dense(nn_count, activation='relu'),
  tf.keras.layers.Dense(out_hours)
])

# I want to calc on float numbers so I use mse and mae for loss and metrics
# Also I have to find out which optimizer is good for me.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='mse',
              metrics=['mae', 'mse'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1100)              0         
_________________________________________________________________
dense (Dense)                (None, 6400)              7046400   
_________________________________________________________________
dense_1 (Dense)              (None, 24)                153624    
Total params: 7,200,024
Trainable params: 7,200,024
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [0]:
train_np = train.to_numpy()
test_np = test.to_numpy()
input_size = len(train_np) - in_hours
print(input_size)

def generate_data():
  while (1):
    for i in range(0, input_size + 1):
      x = train_np[i:in_hours+i]
      x = x[np.newaxis, :, :]
      y = test_np[i:out_hours+i]
      y = y.transpose()
      yield (x, y)
     
model.fit_generator(generate_data(), steps_per_epoch=input_size, epochs=100)

2026
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
2026/2026 [==============================] - 66s 32ms/step - loss: 5241390.0081 - mean_absolute_error: 484.1494 - mean_squared_error: 5241398.5000
Epoch 2/100
2026/2026 [==============================] - 67s 33ms/step - loss: 22202.2884 - mean_absolute_error: 134.4943 - mean_squared_error: 22202.2754
Epoch 3/100
2026/2026 [==============================] - 67s 33ms/step - loss: 16840.8645 - mean_absolute_error: 115.8517 - mean_squared_error: 16840.8652
Epoch 4/100
2026/2026 [==============================] - 67s 33ms/step - loss: 17352.8991 - mean_absolute_error: 121.2243 - mean_squared_error: 17352.9121
Epoch 5/100
2026/2026 [==============================] - 67s 33ms/step - loss: 16627.6465 - mean_absolute_error: 118.9501 - mean_squared_error: 16627.6348
Epoch 6/100
2026/2026 [==============================] - 72s 36ms/step - loss: 16891.0206 - mean_absolute_error: 120.0200 - mean_squared_error: 16891.0098
Epoch 7/100
2

## Now test the model

In [0]:
example_batch = val_in.to_numpy()
example_result = model.predict(example_batch)

import matplotlib.pyplot as plt
ex_b = pd.DataFrame({'price': val_test.price})
ex_r = pd.DataFrame(example_result)

plt.figure(figsize = (12,8))
plt.plot(ex_r.index, ex_b,label = 'Real')
plt.plot(ex_r.index, ex_r, label = 'Prediction')
plt.legend(loc = 'best')
plt.title('BTC price prediction')